# GENERATE PASSENGER CURBSIDE DEMAND

This module takes the arrival and departure demand along with the mode choice percentage and extracts the appropriate amount of demand from the people demand and converts it to cars. 

```python
def create_sumo_demand_passenger_curbside(people,
              percentOfPassenger=.35,
              stops=stop_dict = {
                "A":['A_top_1','A_top_2','A_top_3','A_bot_1','A_bot_2','A_bot_3'],
                "B":['B_top_1','B_top_2','B_top_3','B_bot_1','B_bot_2','B_bot_3'],
                "C":['C_top_1','C_top_2','C_top_3','C_bot_1','C_bot_2','C_bot_3'],
                "D":['D_depart_1','D_depart_2','D_arrive_1','D_arrive_2','D_service'],
                "E":['E_top_1','E_top_2','E_top_3','E_bot_1','E_bot_2','E_bot_3'],

                },
             end_weight = [.2,.8],
             start_weights = [.225,.225,.275,.275],
             stop_duration = np.random.exponential(20,1000) + np.random.normal(60,5,1000),
                                         ):
```

> - **people:** *DataFrame* - is the people demand for the 
> - **percentOFPassenger** *Float* - perecent of passengers going to curb
> - **stops** *Dictionary* - informs the stops for the passenger vehicles
> - **end_weight** *List* - list of floats that add to 1. Informs the destination of each trip. 
> - **start_weights** *List or Array* - list of floats that add to 1. Informs the origin of each trip.
> - **stop_duration** *List or Array* - distribution of dwell times on the curb



In [ ]:
from __future__ import print_function
import xml.etree.ElementTree as ET
from xml.etree.ElementTree import Element, SubElement, Comment
from lxml import etree
from copy import copy
import os
import inspect
from xml.dom import minidom
import pandas as pd
import numpy as np
import random
import matplotlib.pyplot as plt
import seaborn as sns
import sys
from datetime import datetime
sys.path.insert(0, os.path.abspath('../Generate_SUMO_demand/'))
import DFW_gen_flow as gf
import ipywidgets as widgets
from ipywidgets import interact, interactive, fixed, interact_manual
from ipywidgets import ToggleButtons



In [ ]:
Date = '2018-5-13' 


In [ ]:
folder = "../Example_Files/Demand_CSV"
selected = pd.read_csv("selected_demand.csv")
level = selected.loc[selected['Date'] == Date]['Volume_Category'].item()
file_name = Date + '.' + level +".people.csv"
people = pd.read_csv(os.path.join(folder,file_name),index_col=[0])

people.head()

In [ ]:
def create_sumo_demand_passenger_curbside(people,
                                          level,
                                          percentOfPassenger=.309,
                                          peopleToCars=1.7,
                                          stops= {
        "A":['A_top_1','A_top_2','A_top_3','A_bot_1','A_bot_2','A_bot_3'],
        "B":['B_top_1','B_top_2','B_top_3','B_bot_1','B_bot_2','B_bot_3'],
        "C":['C_top_1','C_top_2','C_top_3','C_bot_1','C_bot_2','C_bot_3'],
        "D":['D_depart_1','D_depart_2','D_arrive_1','D_arrive_2','D_service'],
        "E":['E_top_1','E_top_2','E_top_3','E_bot_1','E_bot_2','E_bot_3'],
        
        },
                                         end_weight = [.2,.8],
                                         start_weights = [.225,.225,.275,.275],
                                         stop_duration = np.random.exponential(20,10000) + np.random.normal(60,5,10000),
                                         ):
    
    end_weight_south = end_weight[::-1]
    
    columns = ['Arrive_A_people','Arrive_B_people','Arrive_C_people','Arrive_D_people',
               'Arrive_E_people','Depart_A_people','Depart_B_people',
               'Depart_C_people','Depart_D_people','Depart_E_people']
    starts = ['South_1', 'South_Plaza', 'North_Plaza', 'North_1']
    ends = ['South_Exit', 'North_Exit']
    routes = Element('routes')
    routes.set('xmlns:xsi','http://www.w3.org/2001/XMLSchema-instance')
    routes.set('xsi:noNamespaceSchemaLocation', 'http://sumo.dlr.de/xsd/routes_file.xsd')
    people['seconds'] = np.array(people.index) * 30 * 60
    count = 1
    
    for column in columns:
        column_string = column.split('_')
        terminal = column_string[1]
        for t,numberOfPeople in enumerate(people[column]):
            numberOfVehicles = round((numberOfPeople/peopleToCars) * percentOfPassenger)
            for i in range(numberOfVehicles):
                time = people['seconds'][t] + round(np.random.uniform(0,1800))
                start = np.random.choice(starts,p=start_weights )
                stop = np.random.choice(stops[terminal])
                if start[0] == "S":
                    p = end_weight_south
                else:
                    p = end_weight
                end = np.random.choice(ends,p=p)
                trip = Element('trip')
                trip.set('id', column + '_passenger_' + str(count))
                trip.set('type', 'passenger')
                trip.set('color', "#bb0000")
                trip.set('depart',str(time))
                trip.set('from',start)
                trip.set('to',end)
                trip.set('departSpeed', "max")
                trip.set('departLane', "best")
                
                count+=1
                routes.append(trip)
                duration = str(np.random.choice(stop_duration))
                
                ET.SubElement(trip,"stop",busStop=stop,duration=duration,parking='true')

    routes[:] = sorted(routes, key=lambda child: (child.tag,float(child.get('depart'))))

    file_name = Date + "." + level+ ".passenger.curb.xml"
    folder = "../Example_Files/TempDemandXML"
    print("Saving to xml: ", file_name)
        
    with open(os.path.join(folder,file_name), 'wb') as f:
        f.write(minidom.parseString(ET.tostring(routes)).toprettyxml(encoding="utf-8"))

In [ ]:
create_sumo_demand_passenger_curbside(people,level)

In [ ]:
plt.hist(np.random.exponential(20,1000) + np.random.normal(60,5,1000),bins=100)
plt.show()